In [2]:
import openmm as mm
import shlex
import subprocess


import matplotlib as mpl
import matplotlib.pyplot as plt

import pdbfixer
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology
import os

from openmm.app import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet


In [3]:
base = '/scratch/htc/fsafarov/structures/8ef5_w_i/charmm-gui/charmm-gui'

#We use psf and crd system because pdb files does not support the 6-digit base system for ATOM ID's, which is in our case utilized for POPC. 
#Input below is from psf and crd from CHARMM-GUI.

psf = CharmmPsfFile(os.path.join(base, 'step5_assembly.psf'))
crd = CharmmCrdFile(os.path.join(base, 'step5_assembly.crd'))

#Input all the parameters from toppar folder of CHARMM-GUI output for all the components of psf and crd.

params = CharmmParameterSet(
    os.path.join(base, 'toppar/top_all36_prot.rtf'),
    os.path.join(base, 'toppar/par_all36m_prot.prm'),
    os.path.join(base, 'toppar/top_all36_na.rtf'),
    os.path.join(base, 'toppar/par_all36_na.prm'),
    os.path.join(base, 'toppar/top_all36_carb.rtf'),
    os.path.join(base, 'toppar/par_all36_carb.prm'),
    os.path.join(base, 'toppar/top_all36_lipid.rtf'),
    os.path.join(base, 'toppar/par_all36_lipid.prm'),
    os.path.join(base, 'toppar/top_all36_cgenff.rtf'),
    os.path.join(base, 'toppar/par_all36_cgenff.prm'),
    os.path.join(base, 'toppar/toppar_all36_moreions.str'),
    os.path.join(base, 'toppar/top_interface.rtf'),
    os.path.join(base, 'toppar/par_interface.prm'),
    os.path.join(base, 'toppar/toppar_all36_nano_lig.str'),
    os.path.join(base, 'toppar/toppar_all36_nano_lig_patch.str'),
    os.path.join(base, 'toppar/toppar_all36_synthetic_polymer.str'),
    os.path.join(base, 'toppar/toppar_all36_synthetic_polymer_patch.str'),
    os.path.join(base, 'toppar/toppar_all36_polymer_solvent.str'),
    os.path.join(base, 'toppar/toppar_water_ions.str'),
    os.path.join(base, 'toppar/toppar_dum_noble_gases.str'),
    os.path.join(base, 'toppar/toppar_ions_won.str'),
    os.path.join(base, 'toppar/cam.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_arg0.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_c36m_d_aminoacids.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_fluoro_alkanes.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_heme.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_na_combined.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_retinol.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_model.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_modify_res.str'),
    os.path.join(base, 'toppar/toppar_all36_na_nad_ppi.str'),
    os.path.join(base, 'toppar/toppar_all36_na_rna_modified.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_sphingo.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_archaeal.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_bacterial.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_cardiolipin.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_cholesterol.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_dag.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_inositol.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_lnp.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_lps.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_mycobacterial.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_miscellaneous.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_model.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_prot.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_tag.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_yeast.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_hmmm.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_detergent.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_ether.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_oxidized.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_glycolipid.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_glycopeptide.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_imlab.str'),
    os.path.join(base, 'toppar/toppar_all36_label_spin.str'),
    os.path.join(base, 'toppar/toppar_all36_label_fluorophore.str'),
    os.path.join(base, '7v7/7v7.rtf'),
    os.path.join(base, '7v7/7v7.prm')
)

/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.0; phase=0.0; per=3> with <DihedralType: k=0.12; phase=0.0; per=3>
  warnings.warn('Replacing dihedral %r with %r' %


In [4]:
psf.setBox(80*unit.angstroms, 80*unit.angstroms, 90.0*unit.angstrom)
#Set up a box

In [5]:
print(psf.topology.getPeriodicBoxVectors())
#check if the pbc is working

(Vec3(x=8.0, y=0.0, z=0.0), Vec3(x=0.0, y=8.0, z=0.0), Vec3(x=0.0, y=0.0, z=9.0)) nm


In [6]:
fixer = pdbfixer.PDBFixer(filename='system_box.pdb')
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens()

In [8]:
print(params.atom_types_int);
#Verifying if all the atoms are parametrized

{-1: <openmm.app.internal.charmm.topologyobjects.AtomType object at 0x7ff8e886cf50>}


In [10]:
system = psf.createSystem(params, nonbondedMethod=app.PME, nonbondedCutoff=1.0 * unit.nanometer, constraints = app.HBonds)
#constraints = app.HBonds::We constraint the Hydrogen bonds so that they won\t stretch during the simulation

In [11]:
integrator_eq = mm.LangevinIntegrator(310.10*unit.kelvin, 1/unit.picoseconds, 2.0*unit.femtoseconds)

In [12]:
system.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 310.10*unit.kelvin))

9

In [14]:
platform = mm.Platform.getPlatformByName('CUDA')

In [15]:
print(mm.Platform.getName(platform))

CUDA


In [16]:
simulation_eq = app.Simulation(psf.topology, system, integrator_eq, platform)

simulation_eq.context.setPositions(crd.positions)

In [17]:
print("First atom position:", crd.positions[0])


positions_check = simulation_eq.context.getState(getPositions=True).getPositions()
print("First atom after setPositions:", positions_check[0])


First atom position: Vec3(x=37.6858657556, y=0.9888895581, z=-18.8703364334) A
First atom after setPositions: Vec3(x=3.7685866355895996, y=0.09888935089111328, z=-1.8870337009429932) nm


In [18]:
# Before minimization
state = simulation_eq.context.getState(getEnergy=True)
print("Potential energy before minimization:", state.getPotentialEnergy())

# Run minimization
simulation_eq.minimizeEnergy()

#maxIterations=500, tolerance=10*unit.kilojoule/(unit.nanometers * unit.mole)


# After minimization
state = simulation_eq.context.getState(getEnergy=True)
print("Potential energy after minimization:", state.getPotentialEnergy())


Potential energy before minimization: 2.5368718818352354e+29 kJ/mol
Potential energy after minimization: 1.9578670457359354e+29 kJ/mol


In [ ]:
#simulation_eq.minimizeEnergy()
#default number of iterations is unlimited. maxiterations = 0 mean unlimited.

In [ ]:
for step in range(15):  # Assuming 50 steps, each representing 1 ns of equilibration
    restraint_force = 5.0 * (50 - step) / 50.0  # Taper from 5 kcal/mol/Å² to 0
    #print(step)
    #print(restraint_force)
    
    # Apply harmonic restraints to protein heavy atoms
    # Restraint application code here (custom based on specific atoms or atom groups)

    simulation_eq.step(100)  # Run for 1 ns (modify based on actual timestep size)

In [ ]:
steps_per_microsecond = int(1e6 / (2.4 * 1e-3))# Number of steps per µs with 2.4 fs timestep
print(steps_per_microsecond)
simulation_eq.reporters.append(app.StateDataReporter('/scratch/htc/fsafarov/traj/output_3.log', 10, step=True, potentialEnergy=True, temperature=True))
simulation_eq.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/trajectory_3.dcd', 10))
